In [1]:
import numpy as np
import matplotlib.pyplot as pyplot
import pandas as pd
import scipy.sparse as sps
%matplotlib inline  
%load_ext Cython

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
import optuna

C:\Users\feder\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# data_train_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv"
# data_target_user_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv"
data_train_path="data_train.csv"
data_target_user_path="data_target_users_test.csv"
data_train = pd.read_csv(data_train_path)
data_target = pd.read_csv(data_target_user_path)

In [3]:
URM_all =  data_train.pivot(index='row', columns='col', values='data').fillna(0)
item_map = {i : item for i, item in enumerate(URM_all.columns)}
user_map = {i : user for i, user in enumerate(data_target["user_id"])}
item_map_inv = {item : i for i, item in item_map.items()}
user_map_inv = {user : i for i, user in user_map.items()}
missing_index = [x for x in range(1,13025) if x not in URM_all.index.tolist()]
add_urm = pd.DataFrame(index = missing_index, columns = URM_all.columns).fillna(0)
URM_all = pd.concat([URM_all, add_urm]).sort_index()
del add_urm
del missing_index
#data_target["user_id"] = data_target["user_id"]
URM_all = URM_all.to_numpy()
URM_all = sps.csr_matrix(URM_all)
URM_all

<13024x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [4]:
test_folds = 3

URM_trains = [None] * test_folds
evaluator_tests = [None] * test_folds
for i in range(0, test_folds):
    URM_trains[i], URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
    evaluator_tests[i] = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2578 (19.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2593 (19.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2551 (19.6%) Users that have less than 1 test interactions


In [7]:
def train_evaluate(optuna_trial):
    topk = optuna_trial.suggest_int("topK", 1, 100)
    shrink = optuna_trial.suggest_float("shrink", 0.0, 30)

    mAP = 0.0
    for i in range(0, test_folds):
        KNNCF = ItemKNNCFRecommender(URM_trains[i], verbose=False)
        KNNCF.fit(topK=topk, shrink=shrink)
        result_df, _ = evaluator_tests[i].evaluateRecommender(KNNCF)
        mAP += result_df["MAP"].values[0]
    mAP /= test_folds
    return mAP

In [8]:
study = optuna.create_study(direction="maximize")
study.optimize(train_evaluate, n_trials=250)

[I 2023-12-11 15:01:06,897] A new study created in memory with name: no-name-2bdc140a-2cc6-4251-b5c5-d770b01012bc


Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 1846.48 column/sec. Elapsed time 12.03 sec
EvaluatorHoldout: Processed 10446 (100.0%) in 10.08 sec. Users per second: 1036
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 1655.55 column/sec. Elapsed time 13.42 sec
EvaluatorHoldout: Processed 10431 (100.0%) in 9.71 sec. Users per second: 1074
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 1790.70 column/sec. Elapsed time 12.41 sec
EvaluatorHoldout: Processed 10473 (100.0%) in 9.53 sec. Users per second: 1099


[I 2023-12-11 15:02:14,439] Trial 0 finished with value: 0.04204340747786775 and parameters: {'topK': 100, 'shrink': 21.22422213321013}. Best is trial 0 with value: 0.04204340747786775.


Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 2068.34 column/sec. Elapsed time 10.74 sec
EvaluatorHoldout: Processed 10446 (100.0%) in 8.91 sec. Users per second: 1172
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 2087.77 column/sec. Elapsed time 10.64 sec
EvaluatorHoldout: Processed 10431 (100.0%) in 8.93 sec. Users per second: 1168
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 1995.59 column/sec. Elapsed time 11.14 sec
EvaluatorHoldout: Processed 10473 (100.0%) in 9.02 sec. Users per second: 1161


[I 2023-12-11 15:03:14,051] Trial 1 finished with value: 0.04242597860664038 and parameters: {'topK': 70, 'shrink': 22.62124861428265}. Best is trial 1 with value: 0.04242597860664038.


...

[I 2023-12-11 18:26:14,683] Trial 247 finished with value: 0.04448032354594587 and parameters: {'topK': 9, 'shrink': 5.9906147225363116}. Best is trial 227 with value: 0.04461487376642267.


Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 1965.48 column/sec. Elapsed time 11.31 sec
EvaluatorHoldout: Processed 10446 (100.0%) in 8.32 sec. Users per second: 1256
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 1989.91 column/sec. Elapsed time 11.17 sec
EvaluatorHoldout: Processed 10431 (100.0%) in 8.26 sec. Users per second: 1263
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 1945.78 column/sec. Elapsed time 11.42 sec
EvaluatorHoldout: Processed 10473 (100.0%) in 8.31 sec. Users per second: 1260


[I 2023-12-11 18:27:13,766] Trial 248 finished with value: 0.04225170868195072 and parameters: {'topK': 100, 'shrink': 7.177385174989602}. Best is trial 227 with value: 0.04461487376642267.


Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 2445.00 column/sec. Elapsed time 9.09 sec
EvaluatorHoldout: Processed 10446 (100.0%) in 7.35 sec. Users per second: 1422
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 2428.29 column/sec. Elapsed time 9.15 sec
EvaluatorHoldout: Processed 10431 (100.0%) in 7.22 sec. Users per second: 1445
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 2451.69 column/sec. Elapsed time 9.06 sec
EvaluatorHoldout: Processed 10473 (100.0%) in 7.48 sec. Users per second: 1401


[I 2023-12-11 18:28:03,212] Trial 249 finished with value: 0.04437544306639379 and parameters: {'topK': 14, 'shrink': 10.990332506126439}. Best is trial 227 with value: 0.04461487376642267.


In [25]:
import time
recommender = ItemKNNCFRecommender(urm_sparse)
recommender.fit(shrink=best_parameters_value["shrink"], topK=best_parameters_value["k"])
n_users_to_test = len(data_target["user_id"])
suggestions = pd.DataFrame(columns = ["user_id", "item_list", "item_scores"])
start_time = time.time()

for user_id in (data_target["user_id"]-1):
    suggestion, suggestion_values = recommender.recommend(user_id, cutoff=10, return_scores=True)
    suggestions.loc[len(suggestions)] = [user_id+1, " ".join([str(item_map[x]) for x in suggestion]), " ".join([str(x) for x in np.flip(np.sort(suggestion_values))[0][:10]])]
    
end_time = time.time()

print("Reasonable implementation speed is {:.2f} usr/sec".format(n_users_to_test/(end_time-start_time)))

ItemKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22222 (100.0%), 1877.27 column/sec. Elapsed time 11.84 sec
Reasonable implementation speed is 619.29 usr/sec


In [9]:
#suggestions.to_csv('outputs/collaborative_item_item.csv', index=False)
study.best_trial
params={'topK': 13, 'shrink': 5.230538112297396}, values=[0.04461487376642267]

FrozenTrial(number=227, state=1, values=[0.04461487376642267], datetime_start=datetime.datetime(2023, 12, 11, 18, 9, 6, 230685), datetime_complete=datetime.datetime(2023, 12, 11, 18, 9, 54, 870493), params={'topK': 13, 'shrink': 5.230538112297396}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'topK': IntDistribution(high=100, log=False, low=1, step=1), 'shrink': FloatDistribution(high=30.0, log=False, low=0.0, step=None)}, trial_id=227, value=None)

In [27]:
suggestions

,user_id,item_list,item_scores
0,1,101 403 515 506 36 977 1546 869 922 318,1.9704008 1.750514 1.5578038 1.5354275 1.49453...
1,2,1095 11 47 28 50 4 12 5 136 6,1.4058849 1.3188555 1.3066249 1.2116627 1.1660...
2,3,59 956 584 2748 857 1281 536 259 2172 648,0.3344119 0.3292961 0.31564894 0.3086834 0.306...
3,4,28 50 249 136 145 5 277 111 363 122,2.1428757 1.9647391 1.6645603 1.5421509 1.5406...
4,5,1570 5138 77 7033 9039 4 238 116 148 20,1.1080153 1.0350269 0.9306607 0.92600083 0.922...
...,...,...,...
10877,13020,6450 6198 6452 7395 6749 7394 4323 12468 8743 ...,6.52352 6.0895486 6.045894 5.6441913 5.591287 ...
10878,13021,6179 6426 7027 6720 6451 6749 13621 20518 2050...,11.405987 10.792806 10.601754 10.198996 10.070...
10879,13022,1668 1446 1411 1674 809 10789 1561 8550 8100 1...,1.6723673 1.6144214 1.6026337 1.5664377 1.3270...
10880,13023,837 1107 1124 706 639 1534 978 1146 1290 828,1.4537846 1.3834087 1.3473324 1.3461387 1.3224...
